In [130]:
import numpy as np
import copy as copy
import pandas  as pd


def topsis(df, wgt, impt):
    
    colmns = df.columns
    india = df.index
    ilen=len(df.index)
    for k in range(1,len(colmns)):
        df[colmns[k]] = pd.to_numeric(df[colmns[k]], downcast="float")

    siqure = []
    for k in colmns[1:]:
        terabyt = 0
        for l in range(ilen):
            terabyt += np.square(float(df.iloc[l][k]))
        siqure.append(terabyt)

    ind=0
    df_list = []
    for k in colmns[1:]:
        lits = []
        for l in range(len(india)):
            a = float(df.iloc[l][k] / np.sqrt(siqure[ind]))
            lits.append(a)

        df_list.append(lits)
        ind+=1

    onew_df = pd.DataFrame()
    
    onew_df[colmns[0]] = df[colmns[0]]


    for val in range(len(df_list)):
        onew_df[colmns[val+1]] = df_list[val]

    df = onew_df

#########
    df_list = []
    ind=0
    for k in colmns[1:]:
        lits = []
        for l in range(ilen):
            a = df.iloc[l][k] * (wgt[ind])
            lits.append(a)
        df_list.append(lits)
        ind+=1

    onew_df = pd.DataFrame()
    onew_df[colmns[0]] = df[colmns[0]]

    for val in range(len(df_list)):
        onew_df[colmns[val+1]] = df_list[val]

    df = onew_df
########
    maxm = ['Ideal Best']
    minm = ['Ideal Worst']

    for ind in colmns[1:]:
        column = df[ind]  
        val_min = column.min()
        minm.append(val_min)
        val_max = column.max()
        maxm.append(val_max)
    ind=1
    for k in impt:
        if k=='-':
            maxm[ind],  minm[ind]=minm[ind], maxm[ind]
        ind+=1
          
    df.loc[ilen] = maxm
    df.loc[len(df.index)] = minm
###################
    simple_max = []
    simple_min = []

    for ind2 in range(len(df.index) - 2):
        max_temp = 0
        min_temp = 0
        for ind1 in colmns[1:]:
            min_temp += np.square(df.iloc[ind2][ind1] - df.iloc[-1][ind1])
            max_temp += np.square(df.iloc[ind2][ind1] - df.iloc[-2][ind1])

        simple_min.append(round(np.sqrt(min_temp),4))
        simple_max.append(round(np.sqrt(max_temp),4))

        
##################
    df = df.iloc[:-2 , :]
    smx_len=len(simple_max)
    smn_len=len(simple_min)
    s_avg = []
    for ind3 in range(smx_len):
        s_avg.append((simple_max[ind3] + simple_min[ind3]))

    cent_per = []
    for i in range(smn_len):
        cent_per.append((simple_min[i]/s_avg[i]))
        
    df = df.assign(Topsis_Score =cent_per)
    # assigning rank to it 
    df['Rank'] = df['Topsis_Score'].rank(ascending = 0)

    king_of_ranks = []
    for ind5 in range(ilen):
        king_of_ranks.append(int(df.loc[ind5]['Rank']))
            
    df.drop(['Rank'], axis = 1)

    df['Rank'] = king_of_ranks
    
    return df


dfd = pd.read_excel('data.xlsx')
dfd.to_csv ("101916056-data.csv", index = None, header=True)
dfd = pd.read_csv("101916056-data.csv")
impt = "+,+,-,+,+"
wgt = [1,1,1,1,1]

result = topsis(dfd, wgt, impt)
result.head(10)

,Fund Name,P1,P2,P3,P4,P5,Topsis_Score,Rank
0,M1,0.351077,0.344401,0.421434,0.322539,0.335992,0.603342,3
1,M2,0.380334,0.402609,0.440304,0.242862,0.269808,0.605929,2
2,M3,0.330180,0.300744,0.301923,0.357781,0.353072,0.453003,7
3,M4,0.326000,0.295893,0.402564,0.324837,0.334925,0.532319,5
4,M5,0.392872,0.426863,0.226442,0.476530,0.451281,0.546190,4
5,M6,0.367795,0.373505,0.408854,0.394555,0.397907,0.653308,1
6,M7,0.275847,0.213431,0.333373,0.374636,0.369084,0.393970,8
7,M8,0.388693,0.417161,0.213862,0.283467,0.281550,0.494058,6


In [127]:
df = pd.read_excel('test.xlsx')
dfd.to_csv ("test1.csv", index = None, header=True)
dfd = pd.read_csv("test1.csv")
dfd

,Att,Price,Storage,Camera,Looks
0,M1,250.0,16.0,12.0,5.0
1,M2,200.0,16.0,8.0,3.0
2,M3,300.0,32.0,16.0,4.0
3,M4,275.0,32.0,8.0,4.0
4,M5,225.0,16.0,16.0,2.0


In [129]:
impact = "-,+,+,+"
weight = [0.25,0.25,0.25,0.25]

ans = topsis(dfd, weight, impact)
ans

,Att,Price,Storage,Camera,Looks,Topsis_Score,Rank
0,M1,0.110702,0.075378,0.107143,0.149404,0.534269,3
1,M2,0.088561,0.075378,0.071429,0.089642,0.308314,5
2,M3,0.132842,0.150756,0.142857,0.119523,0.691686,1
3,M4,0.121772,0.150756,0.071429,0.119523,0.534807,2
4,M5,0.099632,0.075378,0.142857,0.059761,0.401222,4
